# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [398]:
#!g1.1
%pip install torch==1.4.0
%pip install transformers==3.5.0

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [400]:
#!g1.1
#!git clone  https://github.com/sberbank-ai/ru-gpts

In [401]:
#!g1.1
#!mkdir models/

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [402]:
#!g1.1
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python3 ru-gpts/pretrain_transformers.py \
    --output_dir=models/neural_kodd_s2 \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train_s.txt \
    --do_eval \
    --evaluate_during_training \
    --eval_data_file=valid_s.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 10 \
    --logging_steps 50 \
    --save_steps 50 \
    --block_size 2048 \
    --overwrite_output_dir

08/24/2021 14:26:11 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/24/2021 14:26:11 - INFO - filelock -   Lock 139957151189328 acquired on /tmp/xdg_cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
Downloading: 100%|██████████████████████████████| 608/608 [00:00<00:00, 598kB/s]
08/24/2021 14:26:12 - INFO - filelock -   Lock 139957151189328 released on /tmp/xdg_cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
08/24/2021 14:26:13 - INFO - filelock -   Lock 139957015124240 acquired on /tmp/xdg_cache/torch/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc3440d25937fdac74506b73b76ee6e67f778a804756214363fc2a1a66ef.lock
Downloading: 100%|█████████████████████████| 1.

## Evaluate model
### Generate with generate_transformers.py script.

In [ ]:
#!g1.1
!python3 ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/neural_kodd_s2/checkpoint-250 \
    --k=5 \
    --p=0.95 \
    --length=15 \
    --repetition_penalty=5

### Generate mannualy

In [ ]:
#best models
#models/neural_kodd/checkpoint-250 perpl~20
#models/neural_kodd_s/checkpoint-150 perpl~9.5

In [391]:
#!g1.1
import numpy as np
import torch

np.random.seed(42)
torch.manual_seed(42)

In [ ]:
#!g1.1
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = 'models/neural_kodd_s/checkpoint-150'

#init model
tok = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.cuda()

In [ ]:
#!g1.1
#tok.encode('</s>', return_tensors="pt")
print(tok.special_tokens_map)

In [ ]:
#!g1.1
text = "<s>Павел"
inpt = tok.encode(text, return_tensors="pt")
out = model.generate(inpt.cuda(),
                     max_length=30, 
                     repetition_penalty=5.0, 
                     do_sample=True, 
                     top_k=5, 
                     top_p=0.95, 
                     temperature=0.8,
                     stop_token='</n>',
                     length_penalty=0.5,
                     bad_words_ids=[[23819], [6490]]) # Слова "iPhone" и "Ничего" запрещены для генерации
out_decoded = tok.decode(out[0])#.split('\n')
title = out_decoded.split('\n')[0].split('.')[0][3:]
print(title)

In [412]:
#!g1.1
#!g1.1
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

np.random.seed(42)
torch.manual_seed(42)

def generate_title(path_to_model: str, context: str, max_length: int=15):
    
    tokenizer = GPT2Tokenizer.from_pretrained(path_to_model)
    model = GPT2LMHeadModel.from_pretrained(path_to_model)
    model.cuda()
    inpt = tokenizer.encode(context, return_tensors="pt")
    out = model.generate(inpt.cuda(), max_length=max_length, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)
    
    return tokenizer.decode(out[0])#.split('\n')[0].split('.')[0][3:]

In [413]:
#!g1.1
context_text = 'Google'
generate_title(path_to_model = 'models/neural_kodd_s2/checkpoint-250', context=context_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Google запускает сервис для поиска по видео</s>\nIn'